In [25]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
from tqdm.auto import tqdm

In [26]:
import json
import cv2
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import math
import copy
from scipy.spatial import distance
from pathlib import Path
import scipy
import numpy as np

In [28]:
def calc_sin_phi(a, b, c):
    return sqrt((a*a + b*b) / (a*a + b*b + c*c))

def calc_cos_phi(a, b, c):
    return c / sqrt(a*a + b*b + c*c)

def calc_u1(a, b, c):
    return -b / sqrt(a*a + b*b)


def calc_u2(a, b, c):
    return a / sqrt(a*a + b*b)

def get_rotation_matrix(normalized_vector):
    a, b, c = normalized_vector
    cos_phi = calc_cos_phi(a, b, c)
    sin_phi = calc_sin_phi(a, b, c)
    u1 = calc_u1(a, b, c)
    u2 = calc_u2(a, b, c)
    rot_matrix = np.array([
        [cos_phi + u1 * u1 * (1 - cos_phi)  , u1 * u2 * (1 - cos_phi)           , u2 * sin_phi  ,  0],
        [u1 * u2 * (1 - cos_phi)            , cos_phi + u2 * u2 * (1 - cos_phi) , -u1 * sin_phi ,  0],
        [-u2 * sin_phi                      , u1 * sin_phi                      ,      cos_phi  , 0],
        [0                                  , 0                                 , 0             ,  1]])
    return rot_matrix

def do_transform(np_pc_points, transform_matrix):
    ones = np.ones((np_pc_points.shape[0], 1))
    homo_np_pc_points = np.c_[ np_pc_points, ones]
    homo_pc_3d = transform_matrix@(homo_np_pc_points.T)
    homo_pc_3d = homo_pc_3d.T[:, :3]
    
    mean_vector = np.mean(homo_pc_3d, axis=0)
    if mean_vector[2] < 0:
        # upsiding if front face is down side
        homo_pc_3d[:, 2] = - homo_pc_3d[:, 2]
        homo_pc_3d[:, 0] = - homo_pc_3d[:, 0]
        mean_vector = np.mean(homo_pc_3d, axis=0)
    # mean_vector[2] = 0
    homo_pc_3d = homo_pc_3d - mean_vector
    return homo_pc_3d

In [29]:
def read_stl_file(pc_path):
    pc_points = o3d.io.read_triangle_mesh(str(pc_path))
    print("Done pc points")
    np_pc_points = np.asarray(pc_points.vertices)
    return pc_points, np_pc_points


def transform_characters(pc_points, np_pc_points, normalized_vector, dest_dir_path, woodblock_path):
    # label_dict = read_json(img_anno_file_path)
    # point_index_mapping = read_3d_point_index(pc_anno_file_path)
    # pc_file_path_list = list(Path(pc_dir_path).glob("*.stl"))
    transform_matrix = get_rotation_matrix(normalized_vector)
    homo_pc_3d = do_transform(np_pc_points, transform_matrix)
    print("Done get transform matrix")
    pc_points.vertices = o3d.utility.Vector3dVector(homo_pc_3d)
    pc_points = o3d.geometry.TriangleMesh.compute_triangle_normals(pc_points)
    pc_points.remove_duplicated_vertices()
    o3d.io.write_triangle_mesh(f'{dest_dir_path}/{woodblock_path.stem}_r.stl', pc_points)

In [30]:
normalized_vector = np.array([ 0.0043585 , -0.03693331, -1.        ])
# normalized_vector = np.array([0, 1, 0])
character_stl = Path("./example.stl")
dest_dir_path = "."

In [31]:
pc_points, np_pc_points = read_stl_file(str(character_stl))

Done pc points


In [32]:
print("Done reading pc points")
transform_characters(pc_points, np_pc_points, normalized_vector, dest_dir_path, character_stl)

Done reading pc points
Done get transform matrix
